## Set up for Google Colab 

In [ ]:
from io import StringIO
from scipy.io import mmread
from scipy.io import mminfo
from google.colab import drive
drive.mount("/content/drive")
datAddr = "/content/drive/MyDrive/CMSC818J_Test/"
from scipy.sparse import csr_matrix
from scipy import sparse
import numpy as np
import sys
import os
import csv
import math

Mounted at /content/drive


## Baseline Architecture 
 

### Assumptions

1.   Each multiplication and accumulation takes 4 cycles
2.   Reading the rows from memory takes 4 cycles
3.   Storing the output takes 2 cycle 
4. PEs can read, compute and write back in parallel





In [ ]:
# Adds the same number of idle cycles to each of PEs
def add_idle_cycles (cycles_num):
  for i in range(total_PEs):
    PE_stats[i][IDLE_CYCLES] += cycles_num 

In [ ]:
# Adds the different number of idle cycles to each of PEs by subtracting the 
# max_non_per_PE which represents the cycles for the slowest PEs from the busy 
# cycles of the other PEs once after all the allocations are finished. 
def add_idle_cycles2(idle_cycle_list, max_nonzero_per_PE, PE_stats):
  for i in range(total_PEs):
    PE_stats[i][IDLE_CYCLES] += max_nonzero_per_PE - idle_cycle_list[i] 
  return PE_stats

In [ ]:
# Simulates PE MAC operations
def PE_mult(output_col_num, PE_num):
  total_cycles_spent = 0 
  PE_to_process = PE_lst.pop(0)
  row_num = PE_output_index.pop(0) 
  accumulation = 0

  # optimization for inner product: whichever dictionary has smallest non-zeros, we iterate over that. 
  if len(B_dict) < len(PE_to_process):
    for col,data in B_dict.items():
      if col in PE_to_process:
        # multiply and then accumulate  
        accumulation += PE_to_process[col] * data 
        PE_stats[PE_num][BUSY_CYCLES] += CYCLES_TO_MUL_AND_ACCUMULATE
        total_cycles_spent += 1 
    output_mat2[row_num][output_col_num] = accumulation 
  else:
    for col,data in PE_to_process.items():
      if col in B_dict:
        # multiply and then accumulate  
        accumulation += B_dict[col] * data 
        PE_stats[PE_num][BUSY_CYCLES] += CYCLES_TO_MUL_AND_ACCUMULATE
        total_cycles_spent += 1 
    output_mat2[row_num][output_col_num] = accumulation 
  
  # calculate max and min non-zero per PEs 
  if len(PE_to_process) > PE_stats[PE_num][MAX_NUM_NON_ZEROS]: 
      PE_stats[PE_num][MAX_NUM_NON_ZEROS] = len(PE_to_process)
  elif len(PE_to_process) < PE_stats[PE_num][MIN_NUM_NON_ZEROS]: 
      PE_stats[PE_num][MIN_NUM_NON_ZEROS] = len(PE_to_process)
  return total_cycles_spent




In [ ]:
## ENUMS for PE_stats
BUSY_CYCLES = 1
MAX_NUM_NON_ZEROS = 2
MIN_NUM_NON_ZEROS = 3
IDLE_CYCLES = 4
CYCLE_TO_READ = 4
CYCLES_TO_MUL_AND_ACCUMULATE = 6 # per element multiplication
WRITE_BACK_RATE = 1000 # can write 1000 elements at a time

In [ ]:
# Checks whether the busy and idle cycles add up to the total number of 
# cycles for each of the output files.  
def check_cycle_accuracy(filename):
  for pe in PE_stats:
    if pe[BUSY_CYCLES] + pe[IDLE_CYCLES] != TOTAL_NUMBER_OF_CYCLES:
      print(filename + " Doesn't match up" + str(total_PEs))


In [ ]:
 # Read the rows from matrices and store them to allocate to PEs
def sort_computer_rows(data_index_A, col_index_A, pointer_index_A, output_row_index):
  PE_rows_read = dict() 
  PE_rows_index = dict()


 # read the rows from memory 
  for i in range(total_PEs):
    if pointer_index_A < mat_csrA_len:
      numOfElems = (mat_csr.indptr[pointer_index_A + 1] - mat_csr.indptr[pointer_index_A])
      data_row_A = mat_csr.data[data_index_A:numOfElems+data_index_A]
      row_A = mat_csr.indices[col_index_A:numOfElems+data_index_A]
      PE_lst.append(dict(zip(row_A, data_row_A)))
      PE_output_index.append(output_row_index)
      data_index_A += numOfElems
      col_index_A += numOfElems
      pointer_index_A += 1
      output_row_index += 1
    else:
      break
       
  return (data_index_A, col_index_A, pointer_index_A, output_row_index)

In [ ]:
# Resets the cycle based on 
def reset_cycle_list(lst):
  for i in range(len(lst)):
    lst[i] = 0

In [ ]:
# Read the matrices and multiply them by each other 
directory = os.fsencode(datAddr)

for pe_num in range(2, 10, 2):
  with open('/content/drive/MyDrive/CMSC818J_Data/Baseline_PE' + str(pe_num) + '.csv', 'w', newline='') as file:
    writer = csv.writer(file)    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        print(filename)

        ## Read the file 
        m = mmread(datAddr+filename)
        mat_csr = m.tocsr()
        mat_csrB = m.tocsc()

        ## Init for PE stats 
        total_PEs = pe_num # number of PEs available. Can change this number as required
        PE_stats = []
        for i in range(total_PEs):
          PE_stat = dict()
          PE_stat[BUSY_CYCLES] = 0  
          PE_stat[MAX_NUM_NON_ZEROS] = 0 
          PE_stat[MIN_NUM_NON_ZEROS ] = sys.maxsize
          PE_stat[IDLE_CYCLES] = 0
          PE_stats.append(PE_stat)
        PE_stats

        max_nonzero_per_PE = 0 # keeps track of max number of nonzero per PE in one allocation cycle
        idle_cycle_list = [] # keeps track of idle cycles for each PE per allocation
        accum_cycle_list = []

        for i in range(total_PEs):
          idle_cycle_list.append(0)
          accum_cycle_list.append(0)


        mat_csrB_len = len(mat_csrB.indptr) - 1
        mat_csrA_len = len(mat_csr.indptr) - 1
        output_mat_len = len(mat_csrB.indptr) - 1
        PE_num = 0
        TOTAL_NUMBER_OF_CYCLES = 0

        TOTAL_NUMBER_OF_CYCLES += CYCLE_TO_READ # For reading elements, one time cost rest of them are amortized with PE computations
        add_idle_cycles(CYCLE_TO_READ)
        # acts as the buffer that stores rows to be allocated to PEs
        PE_lst = []
        PE_output_index = []
        # matrix to store the output 
        output_mat2 = [[0 for _ in range(output_mat_len)] for _ in range(output_mat_len)]
        data_index_B = 0 
        row_index_B = 0
        pointer_index_B = 0 
        output_col_index = 0
        output_row_index = 0


        for j in range(mat_csrB_len): 
          # Decode col of matrix B from csc format
          numOfElems_B = (mat_csrB.indptr[pointer_index_B + 1] - mat_csrB.indptr[pointer_index_B])
          data_row_B = mat_csrB.data[data_index_B:numOfElems_B+data_index_B]
          col_B = mat_csrB.indices[row_index_B:numOfElems_B+data_index_B]

          # The decoded cols and corresponding data values are zipped into a dictionary 
          # data structure to easily pattern match with rows when computing inner product
          B_dict = dict(zip(col_B, data_row_B))
          data_index_B += numOfElems_B
          row_index_B += numOfElems_B
          pointer_index_B += 1
          data_index_A = 0
          col_index_A = 0
          pointer_index_A = 0
          output_row_index = 0
          for i in range(mat_csrA_len):
            # Decode rows of matrix A from csr format 
            if not PE_lst:
              data_index_A, col_index_A, pointer_index_A, output_row_index = 
              sort_computer_rows(data_index_A, col_index_A, pointer_index_A, output_row_index)
            # allocate it to a PE using round-robin strategy 
            total_computation_cycles = PE_mult(output_col_index, PE_num)

            # Calculate the idle cycles after each finishing each allocation 
            # idle_cycle_list keeps track of number of busy cycles per allocation for 
            # each PE. At the end, the PE with the max busy cycle is used to calculate 
            # idle cycles for other PEs. The add_idle_cycles2 computes the number of cycles 
            # each PE has to wait for the slowest PE to finish computation. 
            # The accum_cycle_list keeps track of the number of idle cycles for each PE 
            # while the output is being written back to output buffer. 
            if PE_num != total_PEs - 1:
              max_nonzero_per_PE = max(max_nonzero_per_PE, total_computation_cycles)
              idle_cycle_list[PE_num] = total_computation_cycles * CYCLES_TO_MUL_AND_ACCUMULATE
              accum_cycle_list[PE_num] = math.ceil(total_computation_cycles/WRITE_BACK_RATE)
              PE_stats[PE_num][IDLE_CYCLES] += math.ceil(total_computation_cycles/WRITE_BACK_RATE)
              if i == mat_csrA_len - 1:
                TOTAL_NUMBER_OF_CYCLES += max_nonzero_per_PE * CYCLES_TO_MUL_AND_ACCUMULATE 
                TOTAL_NUMBER_OF_CYCLES += math.ceil(max_nonzero_per_PE/WRITE_BACK_RATE)
                PE_stats = add_idle_cycles2(idle_cycle_list, max_nonzero_per_PE * CYCLES_TO_MUL_AND_ACCUMULATE, PE_stats)
                PE_stats = add_idle_cycles2(accum_cycle_list, math.ceil(max_nonzero_per_PE/WRITE_BACK_RATE), PE_stats)
                max_nonzero_per_PE = 0  
                reset_cycle_list(accum_cycle_list)
                reset_cycle_list(idle_cycle_list)
            else:
              max_nonzero_per_PE = max(max_nonzero_per_PE, total_computation_cycles)
              PE_stats[PE_num][IDLE_CYCLES] += math.ceil(total_computation_cycles/WRITE_BACK_RATE)
              accum_cycle_list[PE_num] = math.ceil(total_computation_cycles/WRITE_BACK_RATE)
              idle_cycle_list[PE_num] = total_computation_cycles * CYCLES_TO_MUL_AND_ACCUMULATE
              TOTAL_NUMBER_OF_CYCLES += max_nonzero_per_PE * CYCLES_TO_MUL_AND_ACCUMULATE 
              TOTAL_NUMBER_OF_CYCLES += math.ceil(max_nonzero_per_PE/WRITE_BACK_RATE)
              PE_stats = add_idle_cycles2(idle_cycle_list, max_nonzero_per_PE * CYCLES_TO_MUL_AND_ACCUMULATE, PE_stats)
              PE_stats = add_idle_cycles2(accum_cycle_list, math.ceil(max_nonzero_per_PE/WRITE_BACK_RATE), PE_stats)
              max_nonzero_per_PE = 0  
              reset_cycle_list(accum_cycle_list)
              reset_cycle_list(idle_cycle_list)
            PE_num = (PE_num + 1) % total_PEs 
          output_col_index += 1 

        # reshape the output matrix so that it can validated 
        output_mat2_reshape = np.reshape(output_mat2, (output_mat_len, output_mat_len))
        correct_output = np.dot(m.todense(),m.todense())
        # Validates if matrix multiplication is done correctly
        print(np.allclose(output_mat2_reshape, correct_output))
        check_cycle_accuracy(filename)
        # prints the number of cycles per PE, max and min number of zeros per PE
        string = [filename, TOTAL_NUMBER_OF_CYCLES]
        for pe in range(pe_num):
          for stat_num in range(1, 5, 1):
            string.append(PE_stats[pe][stat_num])
        # write back the result to the file 
        writer.writerow(string)
    

In [ ]:
# Read the matrices and multiply them by a randomly generated matrix
directory = os.fsencode(datAddr)
for pe_num in range(2, 10, 2):

  with open('/content/drive/MyDrive/CMSC818J_Data/BaselineAB_PE' + str(pe_num) + '.csv', 'w', newline='') as file:
    writer = csv.writer(file)    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        print(filename)

        ## Read the file 
        m = mmread(datAddr+filename)
        m2 = mmread("/content/drive/MyDrive/psmigr_3.mtx") # serves as the random matrix 
        mat_csr = m.tocsr()
        mat_csrB = m2.tocsc()
        mat_csrB.resize(mat_csr.shape) # reshape so that it can be multiplied to first matrix

        ## Init for PE stats 
        total_PEs = pe_num # number of PEs available. Can change this number as required
        PE_stats = [] # keeps track of metrics 
        for i in range(total_PEs):
          PE_stat = dict()
          PE_stat[BUSY_CYCLES] = 0  
          PE_stat[MAX_NUM_NON_ZEROS] = 0 
          PE_stat[MIN_NUM_NON_ZEROS ] = sys.maxsize
          PE_stat[IDLE_CYCLES] = 0
          PE_stats.append(PE_stat)
        PE_stats

        max_nonzero_per_PE = 0 # keeps track of max number of nonzero per PE in one allocation cycle
        idle_cycle_list = [] # keeps track of idle cycles for each PE per allocation
        accum_cycle_list = []

        for i in range(total_PEs):
          idle_cycle_list.append(0)
          accum_cycle_list.append(0)


        mat_csrB_len = len(mat_csrB.indptr) - 1
        mat_csrA_len = len(mat_csr.indptr) - 1
        # output matrix size
        output_mat_lenX = len(mat_csr.indptr) - 1
        output_mat_lenY = len(mat_csrB.indptr) - 1
        PE_num = 0
        TOTAL_NUMBER_OF_CYCLES = 0

        TOTAL_NUMBER_OF_CYCLES += CYCLE_TO_READ # For reading elements, one time cost rest of them are amortized with PE computations
        add_idle_cycles(CYCLE_TO_READ)
        # acts as the buffer that stores rows to be allocated to PEs
        PE_lst = []
        PE_output_index = []
        # matrix to store the file output 
        output_mat2 = np.zeros((output_mat_lenX, output_mat_lenY))
        data_index_B = 0 
        row_index_B = 0
        pointer_index_B = 0 
        output_col_index = 0
        output_row_index = 0


        for j in range(mat_csrB_len): 
          # Decode col of matrix B from csc format
          numOfElems_B = (mat_csrB.indptr[pointer_index_B + 1] - mat_csrB.indptr[pointer_index_B])
          data_row_B = mat_csrB.data[data_index_B:numOfElems_B+data_index_B]
          col_B = mat_csrB.indices[row_index_B:numOfElems_B+data_index_B]

          # The decoded cols and corresponding data values are zipped into a dictionary 
          # data structure to easily pattern match with rows when computing inner product
          B_dict = dict(zip(col_B, data_row_B))
          data_index_B += numOfElems_B
          row_index_B += numOfElems_B
          pointer_index_B += 1
          data_index_A = 0
          col_index_A = 0
          pointer_index_A = 0
          output_row_index = 0
          for i in range(mat_csrA_len):
            # Decode rows of matrix A from csr format 
            if not PE_lst:
              data_index_A, col_index_A, pointer_index_A, output_row_index = sort_computer_rows(data_index_A, col_index_A, pointer_index_A, output_row_index)
            # allocate it to a PE using round-robin strategy 
            total_computation_cycles = PE_mult(output_col_index, PE_num)

            # Calculate the idle cycles after each finishing each allocation 
            # idle_cycle_list keeps track of number of busy cycles per allocation for 
            # each PE. At the end, the PE with the max busy cycle is used to calculate 
            # idle cycles for other PEs. The add_idle_cycles2 computes the number of cycles 
            # each PE has to wait for the slowest PE to finish computation. 
            # The accum_cycle_list keeps track of the number of idle cycles for each PE 
            # while the output is being written back to output buffer. 
            if PE_num != total_PEs - 1:
              max_nonzero_per_PE = max(max_nonzero_per_PE, total_computation_cycles)
              idle_cycle_list[PE_num] = total_computation_cycles * CYCLES_TO_MUL_AND_ACCUMULATE
              accum_cycle_list[PE_num] = math.ceil(total_computation_cycles/WRITE_BACK_RATE)
              PE_stats[PE_num][IDLE_CYCLES] += math.ceil(total_computation_cycles/WRITE_BACK_RATE)
              if i == mat_csrA_len - 1:
                TOTAL_NUMBER_OF_CYCLES += max_nonzero_per_PE * CYCLES_TO_MUL_AND_ACCUMULATE 
                TOTAL_NUMBER_OF_CYCLES += math.ceil(max_nonzero_per_PE/WRITE_BACK_RATE)
                PE_stats = add_idle_cycles2(idle_cycle_list, max_nonzero_per_PE * CYCLES_TO_MUL_AND_ACCUMULATE, PE_stats)
                PE_stats = add_idle_cycles2(accum_cycle_list, math.ceil(max_nonzero_per_PE/WRITE_BACK_RATE), PE_stats)
                max_nonzero_per_PE = 0  
                reset_cycle_list(accum_cycle_list)
                reset_cycle_list(idle_cycle_list)
            else:
              max_nonzero_per_PE = max(max_nonzero_per_PE, total_computation_cycles)
              PE_stats[PE_num][IDLE_CYCLES] += math.ceil(total_computation_cycles/WRITE_BACK_RATE)
              accum_cycle_list[PE_num] = math.ceil(total_computation_cycles/WRITE_BACK_RATE)
              idle_cycle_list[PE_num] = total_computation_cycles * CYCLES_TO_MUL_AND_ACCUMULATE
              TOTAL_NUMBER_OF_CYCLES += max_nonzero_per_PE * CYCLES_TO_MUL_AND_ACCUMULATE 
              TOTAL_NUMBER_OF_CYCLES += math.ceil(max_nonzero_per_PE/WRITE_BACK_RATE)
              PE_stats = add_idle_cycles2(idle_cycle_list, max_nonzero_per_PE * CYCLES_TO_MUL_AND_ACCUMULATE, PE_stats)
              PE_stats = add_idle_cycles2(accum_cycle_list, math.ceil(max_nonzero_per_PE/WRITE_BACK_RATE), PE_stats)
              max_nonzero_per_PE = 0  
              reset_cycle_list(accum_cycle_list)
              reset_cycle_list(idle_cycle_list)
            PE_num = (PE_num + 1) % total_PEs 
          output_col_index += 1 

        # reshape the output matrix so that it can validated 
        output_mat2_reshape = np.reshape(output_mat2, (output_mat_lenX, output_mat_lenY))
        correct_output = (mat_csr * mat_csrB).todense()   
        # Validates if matrix multiplication is done correctly
        print(np.allclose(output_mat2_reshape, correct_output))
        check_cycle_accuracy(filename)
        # prints the number of cycles per PE, max and min number of zeros per PE
        string = [filename, TOTAL_NUMBER_OF_CYCLES]
        for pe in range(pe_num):
          for stat_num in range(1, 5, 1):
            string.append(PE_stats[pe][stat_num])
        # write back the result to the file 
        writer.writerow(string)